In [1]:
import numpy as np
import cv2
import os
import uuid
import math
import serial
import time
import pyrealsense2 as rs
import time
from math import sin, cos, radians

In [42]:
#Create a guids for the 
idmodel = str(uuid.uuid4())

#Gcode path
gcodepath = "C:\\gcode\\bgroot.gcode"
#gcodepath = "C:\\gcode\\tode2.gcode"
#gcodepath = "C:\\gcode\\Rubber_Duck.gcode"
#gcodepath = "C:\\gcode\\cooutbreak_1.gcode"

#Save directory
imagesavedir = "C:\\gcode\\" + idmodel + "\\"
os.mkdir(imagesavedir)

#Printbed filename
printbedimg = "C:\\gcode\\topbed_2.jpg"

In [3]:
#Get a list of coordinates
def getcoordinates(gcode, multiply):
    data = gcode.split()
    #print("D", data)
    cmd = data[0]
    z=0
    if cmd == "G0" or cmd == "G1":
        for d1 in data:
            if d1[0:1].lower() == "x":
                x = float(d1[1:]) * multiply
            if d1[0:1].lower() == "y":
                y = float(d1[1:]) * multiply
            if d1[0:1].lower() == "z":
                z = float(d1[1:]) * multiply
        return x,y,z
    else:
        return 0,0,0

In [79]:
#Appendimg
def appendimg(xstart, ystart, xend, yend, img1, pixel, color, anglex, angley, rotation, imagesize):
    b = 0
    g = 0
    r = 0
    if color < 256:
        b = color
    if color > 255 and color < 511:
        g = color - 255
        b = 255
    if color > 510:
        r = color - 510
        b = 255
        g = 255
    theta = radians(rotation)
    cosang, sinang = cos(theta), sin(theta)
    cx = imagesize / 2
    cy = imagesize / 2
    
    x1 = xstart
    x2 = xend
    y1 = ystart
    y2 = yend
    
    tx1, ty1 = x1-cx, y1-cy
    p1x = ( tx1*cosang + ty1*sinang) + cx
    p1y = (-tx1*sinang + ty1*cosang) + cy
    tx2, ty2 = x2-cx, y2-cy
    p2x = ( tx2*cosang + ty2*sinang) + cx
    p2y = (-tx2*sinang + ty2*cosang) + cy
        
    #pts = np.array([[xstart+angle,ystart-angle],[xend+angle,yend-angle]], np.int32)
    pts = np.array([[p1x+anglex,p1y-angley],[p2x+anglex,p2y-angley]], np.int32)

    #print("Line " + str(xend))
    pts = pts.reshape((-1,1,2))
    img1 = cv2.polylines(img1,[pts],True,(0,0,0), pixel)
    #img1 = cv2.polylines(img1,[pts],True,(b,g,r), pixel) 
    ptslst = [p1x+anglex,p1y-angley,p2x+anglex,p2y-angley]
    return img1, ptslst

In [5]:
#Get Image - If use Realsense
def GetImage(savepath):
    t_end = time.time() + 10

    # Configure depth and color streams
    pipeline = rs.pipeline()
    config = rs.config()
    config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
    config.enable_stream(rs.stream.color, 1920, 1080, rs.format.bgr8, 30)
    
    # Start streaming
    profile = pipeline.start(config)
    
    try:
        while time.time() < t_end:
    
            # Wait for a coherent pair of frames: depth and color
            frames = pipeline.wait_for_frames()
            depth_frame = frames.get_depth_frame()
            color_frame = frames.get_color_frame()
            if not depth_frame or not color_frame:
                continue
    
            # Convert images to numpy arrays
            depth_image = np.asanyarray(depth_frame.get_data())
            color_image = np.asanyarray(color_frame.get_data())
            filename = savepath + "\\RealSenseImg_" + str(1) + ".png"
            cv2.imwrite(filename, color_image)
    finally:
        # Stop streaming
        pipeline.stop()
        return filename, color_image

In [6]:
#Get angles - connected a MPU-6050 to a Arduino - Add Arduino code for this one
def getangles():
    t_end = time.time() + 10
    arduino = serial.Serial('COM4', 9600, timeout=.1)
    test = "dd"
    DegX = 0.0
    DegY = 0.0
    DegZ = 0.0
    while time.time() < t_end:
        data = arduino.readline()[:-2].decode("utf-8") #the last bit gets rid of the new-line chars
        if data:
            print(data)
            if "AngleX" in data:
                data = data[8:]
                DegX = data
            if "AngleY" in data:
                data = data[8:]
                DegY = data
            if "AngleZ" in data:
                data = data[8:]
                DegZ = data
    arduino.close()
    DegX = float(DegX)
    DegY = float(DegY)
    DegZ = float(DegZ)
    DegX = int(DegX)
    DegY = int(DegY)
    DegZ = int(DegZ)
    return DegX, DegY, DegZ

In [7]:
#Parse gcode
def parsegcode():
    with open(gcodepath, "r") as f:
        gcodes = [line.rstrip('\n') for line in f]
    nlayers = 0
    for txtline in gcodes:
        hasg0p = "G0" in txtline
        haszp = "Z" in txtline
        if haszp and hasg0p:
            nlayers = nlayers + 1
    print("Number of layers " + str(nlayers))
    nlayers = nlayers - 1
    layerbatch = int(nlayers/10)
    print("Layer batch: " + str(layerbatch))
    return nlayers, layerbatch, gcodes

In [8]:
#Resize Img
def imageresize(img, to, filename):
    #Resize
    width = int((to/img.shape[0])*img.shape[1])
    print(width)
    height = to
    dim = (width, height)
    # resize image
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    
    #Crop
    diff = int((width - to) / 2)
    print(diff)
    cropped = resized[0:to, diff:int(width-diff)]
    
    cv2.imwrite(filename, cropped)
    return cropped

In [71]:
#Render out images for each layer of gcode
def gcodelayerpoints(anglexdeg, angleydeg, rotation, imgfilename, nlayers, gcodes, multiply, imagesize, pixel, imagesavedir):
    #img = np.zeros((imagesize,imagesize,3), np.uint8) #<-- if use blank img!.
    #img.fill(255) #<-- if use blank img!.
    img = cv2.imread(imgfilename) #Rem out if use blank
    count = 0
    xstart = 0
    ystart = 0
    xend = 0
    yend = 0
    xstarti = 0
    ystarti = 0
    xendi = 0
    yendi = 0
    layer = True
    hasg0 = False
    hasg1 = False
    dim = (int(imagesize/multiply), int(imagesize/multiply))
    color = 100
    counter = nlayers
    zvalue = 0
    start = False
    pts = np.array([[123,3443],[123,123]], np.int32)
    ptslst = []
    anglex = 0
    angley = 0
    #anglexdeg = 0
    #angleydeg = 44.74
    #rotation = 180
    for txtline in gcodes:
        hasg0 = "G0" in txtline
        hasg1 = "G1"in txtline
        hasx = "X" in txtline
        hasy = "Y" in txtline
        hase = "E" in txtline
        hasz = "Z" in txtline
        if layer:
            #Outer lines
            if hase and hasx and hasy and xstart == 0 and ystart == 0:
                xstart, ystart, zvalue = getcoordinates(txtline, multiply)
            if xstart != 0 and ystart != 0 and hasx and hasy:
                xend, yend, zvalue = getcoordinates(txtline, multiply)
            if xstart != 0 and ystart != 0 and xend != 0 and yend != 0:
                img, pts1 = appendimg(xstart, ystart, xend, yend, img, pixel, color + 3, anglex, angley, rotation, imagesize)
                ptslst.append(pts1)
                if hase:
                    xstart = xend
                    ystart = yend
                else:
                    xstart = 0
                    ystart = 0
                xend = 0
                yend = 0
            #Inner lines
            if hasg1 and hasx and hasy and hase:
                xstarti, ystarti, zvalue = getcoordinates(txtline, multiply)
            if hasg0 and hasx and hasy:
                xendi, yendi, zvalue = getcoordinates(txtline, multiply)
            if xstarti != 0 and ystarti != 0 and xendi != 0 and yendi != 0:
                img, pts2 = appendimg(xstarti, ystarti, xendi, yendi, img, pixel, color + 5, anglex, angley, rotation, imagesize)
                ptslst.append(pts2)
                xstarti = 0
                ystarti = 0
                xendi = 0
                yendi = 0
        if hasz and hasg0:
            if layer:
                count = count + 1
                filenamerall = imagesavedir + "\\" + "gcodelyr_" + str(count) + ".JPG"
                if(count == nlayers):
                    resizedrall = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)  
                    cv2.imwrite(filenamerall, img)
                    print(filenamerall)
                counter = counter + 1
                imglyr = np.zeros((imagesize,imagesize,3), np.uint8)
                imglyr.fill(255)
                color = color + 1
                
                 ## Calculate angle value
                anglex = round(math.sin(math.radians(anglexdeg))*zvalue, 2)
                #print("X" + str(anglex))
                angley = round(math.sin(math.radians(angleydeg))*zvalue, 2)
                #print("Y" + str(angley))
            layer = True
    return ptslst

In [ ]:
#If Use a Arduino + MPU
DegX, DegY, DegZ = getangles()

In [ ]:
print(DegX)

In [ ]:
#If use a Realsense camers
filename, image_1 = GetImage()
print(filename)

In [ ]:
#Resize image
image = imageresize(image_1, 4300, filename)

In [10]:
#Get layer information
nlayers, layerbatch, gcodes = parsegcode()

Number of layers 365
Layer batch: 36


In [ ]:
#Degz = DegZ - 180
#DegY = 360 - DegY
#DegX = 360 - DegX
DegZ = 360 - DegZ

In [96]:
DegX = 7
DegY = 18
DegZ = 0

In [97]:
#DegZ = DegZ+270
multiply = 6
imagesize = 3024
pixel = 1
ptslst = gcodelayerpoints(DegX, DegY, DegZ, printbedimg, nlayers, gcodes, multiply, imagesize, pixel, imagesavedir)

C:\gcode\5b887088-e057-4932-bdc6-f5717e1dde8f\\gcodelyr_364.JPG
